In [1]:
from pairwise_model import *
from text_utils import *
import regex as re
from bm25_utils import BM25Gensim
from qa_model import *
from qa_single import *
from tqdm.auto import tqdm
tqdm.pandas()

C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\transformers\models\auto\tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'graph_tool', 'infomap', 'wurlitzer', 'leidenalg'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'leidenalg'}


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df_wiki_windows = pd.read_csv("./data/wikipedia_20220620_cleaned_v2.csv")
df_wiki = pd.read_csv("./data/wikipedia_20220620_short.csv")
df_wiki.title = df_wiki.title.apply(str)

In [4]:
df_wiki.head()

,title,text
0,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s..."
1,Internet Society,Internet Society\n\nInternet Society hay ISOC ...
2,Tiếng Việt,"Tiếng Việt\n\nTiếng Việt, cũng gọi là tiếng Vi..."
3,Ohio,"Ohio\n\nOhio (viết tắt là OH, viết tắt cũ là O..."
4,California,"California\n\nCalifornia (phát âm như ""Ca-li-p..."


In [5]:
entity_dict = json.load(open("./data/entities.json"))
new_dict = dict()
for key, val in entity_dict.items():
    val = val.replace("wiki/", "").replace("_", " ")
    entity_dict[key] = val
    key = preprocess(key)
    new_dict[key.lower()] = val
entity_dict.update(new_dict)
title2idx = dict([(x.strip(), y) for x, y in zip(df_wiki.title, df_wiki.index.values)])

In [6]:
qa_model = QAModel("./data/phobert_final", entity_dict)
pairwise_model_stage1 = PairwiseModel("nguyenvulebinh/vi-mrc-base").half()
pairwise_model_stage1.load_state_dict(torch.load("./data/pairwise_v2.bin"), strict = False)
pairwise_model_stage1.eval()

pairwise_model_stage2 = PairwiseModel("nguyenvulebinh/vi-mrc-base").half()
pairwise_model_stage2.load_state_dict(torch.load("./data/pairwise_stage2_seed0.bin"), strict = False)

OSError: Can't load tokenizer for './data/phobert_final'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './data/phobert_final' is the correct path to a directory containing all relevant files for a PhobertTokenizer tokenizer.

In [ ]:
bm25_model_stage1 = BM25Gensim("./data/bm25_stage1/", entity_dict, title2idx)
bm25_model_stage2_full = BM25Gensim("./data/bm25_stage2/full_text/", entity_dict, title2idx)
bm25_model_stage2_title = BM25Gensim("./data/bm25_stage2/title/", entity_dict, title2idx)

In [ ]:
def get_answer_e2e(question):
    #Bm25 retrieval for top200 candidates
    query = preprocess(question).lower()
    top_n, bm25_scores = bm25_model_stage1.get_topk_stage1(query, topk=200)
    titles = [preprocess(df_wiki_windows.title.values[i]) for i in top_n]
    texts = [preprocess(df_wiki_windows.text.values[i]) for i in top_n]
    
    #Reranking with pairwise model for top10
    question = preprocess(question)
    ranking_preds = pairwise_model_stage1.stage1_ranking(question, texts)
    ranking_scores = ranking_preds * bm25_scores
    
    #Question answering
    best_idxs = np.argsort(ranking_scores)[-10:]
    ranking_scores = np.array(ranking_scores)[best_idxs]
    texts = np.array(texts)[best_idxs]
    best_answer = qa_model(question, texts, ranking_scores)
    if best_answer is None:
        return "Chịu"
    bm25_answer = preprocess(str(best_answer).lower(), max_length=128, remove_puncts=True)
    
    #Entity mapping
    if not check_number(bm25_answer):
        bm25_question = preprocess(str(question).lower(), max_length=128, remove_puncts=True)
        bm25_question_answer = bm25_question + " " + bm25_answer
        candidates, scores = bm25_model_stage2_title.get_topk_stage2(bm25_answer, raw_answer=best_answer)
        titles = [df_wiki.title.values[i] for i in candidates]
        texts = [df_wiki.text.values[i] for i in candidates]
        ranking_preds = pairwise_model_stage2.stage2_ranking(question, best_answer, titles, texts)
        if ranking_preds.max() >= 0.1:
            final_answer = titles[ranking_preds.argmax()]
        else:
            candidates, scores = bm25_model_stage2_full.get_topk_stage2(bm25_question_answer)
            titles = [df_wiki.title.values[i] for i in candidates] + titles
            texts = [df_wiki.text.values[i] for i in candidates] + texts
            ranking_preds = np.concatenate(
                [pairwise_model_stage2.stage2_ranking(question, best_answer, titles, texts), ranking_preds])
        final_answer = "wiki/"+titles[ranking_preds.argmax()].replace(" ","_")
    else:
        final_answer = bm25_answer.lower()
    return final_answer

In [ ]:
get_answer_e2e("Ai là chủ tịch nước Việt Nam 2018")